## Location optimization for establishing a new Chinese restaurant in Vancouver

### Introduction :
Vancouver city is one of the big cities in Canada which located in the west. Vancouver CSD (Census Subdivision), Vancouver city consists of 22 neighborhoods. Besides, Vancouver city is a multicultural city. It is formed by a mix of people who are different races, having different religions, ethnicities, and cultural.

### Business problem :
There are 167180 Chinese people staying in Vancouver city. Thus, this makes establishing a Chinese restaurant a good choice of investment. As a invester, it is always important to find a optimal place to establish a Chinese restaurant. In this case, the neighborhood of Vancouver CSD need to be scanned through to identify establishing a Chinese restaurant in which area will higher business opportunity and lesser competition. This project is targeted to invester who would like to establish a new Chinese restaurant in Vancouver CSD.

### Data :
In order to establish a Chinese restaurant in Vancouver CSD, we will be scrapping and getting the data from the following :

1) The list of Vancouver CSD neighborhoods' name and its postal codes :-

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V

Example : V6L for Arbutus Ridge neighborhood

2) City of Vancouver census local area profiles 2016 :- 

https://opendata.vancouver.ca/explore/dataset/census-local-area-profiles-2016/information/

Example : There are 3045 Chinese people staying in Arbutus Ridge neighborhood.

3) Latitude and longitude of neighborhood in Vancouver city can be gotten through usage of python library Geocoder. Refer to the following link for more information :-

https://geocoder.readthedocs.io/

4) Foursquare API to explore the famous venues especially Chinese restaurant in the neighborhood of Vancouver CSD. One will need to register her developer account from the following URL in order to access to Foursquare API.

https://developer.foursquare.com/

Example : To explore Arbutus Ridge neighborhood and identify the number of Chinese restaurant.